# Report : Project of Basics of Mobile Robotics 

Group members :

- Cirillo Thomas
- Muller Nathan
- Sahin Ali Fuat
- Vinet Pierre

## Introduction of the environment 

Our environment consist of a white surface with Aruco markers on the different corners to be able to identify them using the webcam given to us. We first decided to use a white surface with a black grid but then decided to go with a fully white surface instead because we use a binary threshold to detect our obstacles which would then interpret the grid as an obstacle. To correct this we would had needed to specifically extract the grid to create a mask out of it and substract it to our image in order to not detect it as an obstacle but we would not need this if we used a simple white surface. We also decided that the obstacle were not just squared one and therefor the use of a grid made less sense. 

The start and the goal are represented as Aruco markers as well. The obstacle used for the global navigation are black objects because as said before we use a binary threshold to detect them and blakc works the best for this situation, and the one used for the local avoidance are white objects because since they have the same color as the surface they are not detected with our image processing program.

The code is structured into different parts :

- Image Processing
- Kalman Filter
- Pathfinding
- Global Navigation
- Local Avoidance

There is a different .py file for every part

## 1. Image Processing

The goal of this section is to detect 4 aruco markers present at the 4 corners of the environment using the webcam given to use for the project. The reason we chose aruco markers is that we wanted markers tht the webcam could detect robustly. We tried using QR codes but we found out afterward that Aruco markers were more easily and consistently detected by the webcam than QR codes. Those were more sensible to light differences causing them to be less effectively detected than Aruco markers who seem more robust against those changes. Once the 4 corners are detected and the image has been correctly warped, it detects and updates the position of the robots and of the goal, thresholding the image to find the global obstacles and creating a grid for the pathinding algorithm to use.

Using the OpenCV library, we connect to the webcam using cv2.VideoCapture() and analyze the frames while the camera is connected

In [ ]:
#Connect to the webcam 
cap = cv2.VideoCapture(0)

#As long as the webcam is connected, the frames are store in the image variable
while cap.isOpened():
		ret, image = cap.read()

Once the camera is connected, it will warp the image giving by the webcam to make the image fit the screen. To do so, it checks the transform_start variable : 
<ul>
   <li> if set to <b>True</b>, it calls the functions detecting Aruco markers which return the different coordinates of the different markers
</ul>

This aruco_read function (given below) takes as argument the image that will be processed and two boolean. 
It checks for Aruco markers on the image. When all the desired markers are detected, the coordinates and ids are written in a list used to draw their contours and center.

It then returns this list and the image.

In [ ]:
def aruco_read(image, transform, start):
	ARUCO_DICT = {
		"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
		"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
		"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
		"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
		"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
		"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
		"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
		"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
		"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
		"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
		"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
		"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
		"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
		"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
		"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
		"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
		"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
		"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
		"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
		"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
		"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
	}

	aruco_dict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT["DICT_4X4_50"])
	aruco_params = cv2.aruco.DetectorParameters()
	aruco_det = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

    #Detect if markers are recognized and stock the different variables in corners, ids and rejected
	(corners, ids, rejected) = aruco_det.detectMarkers(image)
    
    #Check if the ids detected from the markers (if any) are the correct one
	if ids is None:
		return None, image
	elif transform and 0 in ids and 1 in ids and 2 in ids and 3 in ids:
		pass
	elif not transform and start and 4 in ids and 5 in ids:
		pass
	elif not transform and not start and (4 in ids or 5 in ids):
		pass
	else:
		return None, image
    
    #If all the correct markers are detected, put all their coordinates in a list and draw their contour and center
	coord_list = []
	ids = ids.flatten()
    
    #Goes through all the coordinates and ids of the detected markers
	for (marker_corners, marker_ids) in zip(corners, ids):
		corners = marker_corners.reshape((4,2))
		(topLeft, topRight, bottomRight, bottomLeft) = corners
        
        #Put all the coordinates and ids in a list
		coord_list.append({'ID': marker_ids, 'POS': np.squeeze(corners)})			
	
        #Draw the contour of the marker
		topRight = (int(topRight[0]), int(topRight[1]))
		bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
		bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
		topLeft = (int(topLeft[0]), int(topLeft[1]))
		cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
		cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
		cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
		cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
		
        #Draw the center of it
		cX = int((topLeft[0] + bottomRight[0]) / 2.0)
		cY = int((topLeft[1] + bottomRight[1]) / 2.0)
		cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
	return coord_list, image

<ul>
    <li>Finally, if the tranform_start variable is set to <b>False</b>, it calls the function change_perspective whose goal is to warp the desired part of the image to make it fit the screen
</ul>

This function (given below) takes as arguments the image we want to warp, the different coordinates and ids of the Aruco markers and the size of the desired output.

We decided to warp the image because it reduce the amount of unused data the program processes.

The function store the coordinates of the different corners as input coordinates for the warp. It then declare output coordinates based on the size variable declared before.

The way the warping works is that it takes input and output coordinates and warp the given image so that the input coordinate A is now situated at the output coordinate A (therefor, in our code it puts the different corners at the point (0,0), (1000,0), (0,1000), (1000,1000); 1000 being the value for size[0] and size[1])

Finally, the function return the warped image to the main.

In [ ]:
def change_perpective(image, coords, size):
	transform_coords = np.zeros((4,2))
    
    #Goes through all the detected markers to assignated corners coordinate as input coordinates
	for el in coords:
		if el.get('ID') == 0:
			transform_coords[0] = el.get('POS')[2]
		elif el.get('ID') == 1:
			transform_coords[1] = el.get('POS')[3]
		elif el.get('ID') == 2:
			transform_coords[2] = el.get('POS')[1]
		elif el.get('ID') == 3:
			transform_coords[3] = el.get('POS')[0]
		else:
			pass
    
    #Declare corners (input coordinates) and dim (output coordinates)
	corners = np.float32(transform_coords)
	dim = np.float32([[0,0],[size[0],0],[0,size[1]],[size[0],size[1]]])

    #Warp the image
	perspective = cv2.getPerspectiveTransform(corners, dim)
	image = cv2.warpPerspective(image, perspective, (size[0], size[1]))
	
	return image

Once the image has been warped, the same detecting process is done to detect the position of the robot and the goal, both detected with Aruco markers (the detection being made if the movement is starting and also if it has already being launched)

In order to detect the different obstacles once all of the previously stated elements are detected, a threshold is applied to the image. This threshold allows to better identify the global obstacles, the environment being white and the obstacles being black (the function is givenbelow)

In [ ]:
def image_threshold(image, border_size, robot_coords, goal_coords):
    
    #Convert to grayscale
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #Reduce image noise
	blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    #Threshold image to better detect obstacle
	_, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
	
    #Remove robot and goal Aruco marker from contour detections
	thresh = delete_aruco(thresh, robot_coords)
	thresh = delete_aruco(thresh, goal_coords)

	kernel = np.ones((5, 5), np.uint8) 
	thresh = cv2.dilate(thresh, kernel, iterations=2)

    #Detect remaining obstacles
	contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
	cv2.drawContours(thresh, contours, -1, 0, border_size)
	
	return thresh

The image is first converted from BRG to grayscale to accuentuate the contrast between the elements. A gaussian blur is then applied to reduce the noise of the image improveing the obstacle detection. Finally, a threshold is applied to the image. We first only used a binary threshold because it was the most efficient one in our case (black obstacles with white surface) but the sensibility to light was to high so we added an Otsu threshold allowing to reduce the impact lightning had on the detection.

The obstacles, that are polygons are detected with OpenCV. 
However, the Aruco markers used to represent the robot as well as the goal as also polygons, so, the delete_aruco function (given below) has to be called to not detect those two.

The function then returns the thresholded image containing the obstacles.

In [ ]:
def delete_aruco(thresh, pos):
	max_p = [0, 0]
	min_p = [float('inf'), float('inf')]
	for point in pos:
		if point[0] > max_p[0]:
			max_p[0] = int(point[0])
		if point[0] < min_p[0]:
			min_p[0] = int(point[0])
		if point[1] > max_p[1]:
			max_p[1] = int(point[1])
		if point[1] < min_p[1]:
			min_p[1] = int(point[1])
	thresh = cv2.rectangle(thresh, min_p, max_p, 255, -1)
	return thresh

Finally, using the thresholded image containing the obstacles, a grid is created using the define_grid function (given below). This grid being used by the pathfinding algorithm, the spacing between each dot representing 80mm in rality. This spacing can be modified but this settings has be kept because it produces satisfying results.

To create the grid, an empty matrix the size of the image is created and creates a dot everywhere there are no obstacles from the thresholded image.

The function then returns the completed grid as well as the coordinates where there are no obstacles.

In [ ]:
def define_grid(size, spacing, thresh):
	background = np.zeros([size[0], size[1], 1], dtype=np.uint8)

	vert = size[0]//spacing
	horz = size[1]//spacing
	v_blank = (size[0] % spacing)//2 + spacing
	h_blank = (size[1] % spacing)//2 + spacing

	coord_init = (v_blank, h_blank)
	grid = []
	coord = []

	for h in range(horz-1):
		coord_init = (h_blank + h*spacing, v_blank)
		for v in range(vert-1):
			if thresh.T[coord_init] == 255:
				background = cv2.circle(background, coord_init, 5, 255, -1)
				grid.append((h,v))
				coord.append(coord_init)
			coord_init = (coord_init[0], coord_init[1] + spacing)
			
	return grid, coord, background

Finally, a find_pos function is called in the main, returning the corresponding position of the start and the goal on the grid we just created.